In [1]:
import sys
sys.path.append('..')

from selenium import webdriver
from fake_useragent import UserAgent
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import multiprocessing
import json
import time
import os
import math
import argparse
import gc
from crawler.crawler_master import CrawlerMaster
import random

In [2]:
TIME_SLEEP       = 2
TIME_OUT         = 40
MAX_COUNT_LOOP   = 100
PUBLISHER        = 'vnexpress'
RAW_DATA_PATH    = os.path.join('../data/raw_data', PUBLISHER)
KILL_FIREFOX_CMD = "pkill firefox"

In [3]:
class CrawlerMaster():
    def __init__(self):
        self.options = self.set_option_web_driver()
        self.url     = 'https://www.foody.vn/ha-noi'

    def set_option_web_driver(self):
        options    = Options()
        ua         = UserAgent()
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'#ua.random
        options.add_argument(f'user-agent={user_agent}')
        options.add_argument("window-size=1920,1080")
#         options.add_argument('--headless')
        return options

In [4]:
def random_time_sleep(level):
    if level == 0:
        return random.uniform(0.01, 0.5)
    if level == 1:
        return random.uniform(0.5, 1)
    if level >= 2:
        return random.uniform(1, 2)

class CrawlerSlave():
    def __init__(self, url, options):
        self.id       = 0
        self.comments = []
        self.driver   = webdriver.Firefox(executable_path='../web_driver/geckodriver', options=options)
        # set page load timeout
        self.driver.set_page_load_timeout(TIME_OUT)
        try:
            self.driver.get(url)
        except:
            print('Time out')
            self.driver = None
        self.log_in()
        # wait for loading page
        time.sleep(TIME_SLEEP)
        self.page_down(times=15)
        
#         if self.driver != None:
#             self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight)')
    
    def log_in(self):
        log_in_button = self.driver.find_element_by_class_name('account-manage')
        log_in_button.click()
        time.sleep(random_time_sleep(level=1))
        user_name_block = self.driver.find_element_by_xpath('//*[@id="Email"]')
        user_name_block.send_keys('langtunhi96@gmail.com')
        time.sleep(random_time_sleep(level=1))
        pass_word_block = self.driver.find_element_by_xpath('//*[@id="Password"]')
        pass_word_block.send_keys('langtunhi96')
        pass_word_block.submit()
        time.sleep(random_time_sleep(level=2))
    
    def page_down(self, times):
        body = self.driver.find_element_by_css_selector('body')
        for i in range(times):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.1)
    
    def load_more_result(self):
        load_more_button = self.driver.find_element_by_class_name('fd-btn-more')
        while True:
            try:
                load_more_button.click()
                self.page_down(times=2)
            except:
                break    
    
    def get_comment_info(self, root_element, id):
        comment_info = {
            'id'         : id,
            'content'    : "",
            'user_name'  : "",
            'like_count' : 0
        }
        # get content of comment
        try:
            content_less = root_element.find_element_by_class_name('content_less')
        except:
            pass

In [5]:
master = CrawlerMaster()
slave = CrawlerSlave(url=master.url, options=master.options)
slave.load_more_result()